# ΕΠΕΞΕΡΓΑΣΙΑ ΦΥΣΙΚΗΣ ΓΛΩΣΣΑΣ - Εργασία 2
## Γ. Text Classification with RNNs
**Μάθημα:**  Επεξεργασία Φυσικής Γλώσσας 

**Συγγραφέας:** Ιωάννης Kουτσούκης 

**Εκδόσεις:** 2025-04-25 | v.0.0.1

# 🧱 Προετοιμασία Περιβάλλοντος

Το περιβάλλον υλοποιήθηκε σε **Mac με επεξεργαστή Apple Silicon (M1/M2/M3)**, με στόχο την αξιοποίηση των διαθέσιμων **υπολογιστικών πυρήνων GPU ή Neural Engine** μέσω του `torch.backends.mps`.

Για λόγους συμβατότητας και αποφυγής σφαλμάτων, επιλέχθηκαν οι **συγκεκριμένες εκδόσεις** των βιβλιοθηκών `torch` και `torchtext`. Η χρήση παλαιότερων ή νεότερων εκδόσεων ενδέχεται να οδηγήσει σε runtime conflicts, ειδικά σε Apple-based συστήματα.


In [155]:
%pip install datasets numpy matplotlib --quiet

Note: you may need to restart the kernel to use updated packages.


In [ ]:
%pip install numpy==1.24.4 --force-reinstall --quiet


  Using cached numpy-1.24.4-cp311-cp311-macosx_11_0_arm64.whl.metadata (5.6 kB)
Using cached numpy-1.24.4-cp311-cp311-macosx_11_0_arm64.whl (13.8 MB)
  Attempting uninstall: numpy
    Found existing installation: numpy 1.24.4
    Uninstalling numpy-1.24.4:
      Successfully uninstalled numpy-1.24.4
Note: you may need to restart the kernel to use updated packages.


In [156]:
%pip install torch==2.1.0 torchtext==0.16.0  --quiet # οι υπολοιπες εκδοσεις εβγαζαν σφαλματα ασυμβατοτητας

Note: you may need to restart the kernel to use updated packages.


In [158]:
import torch

if torch.backends.mps.is_available():
    device = torch.device("mps")
    print("🔥 Χρησιμοποιείται συσκευή: Apple Silicon GPU (MPS)")
elif torch.cuda.is_available():
    device = torch.device("cuda")
    print("⚡ Χρησιμοποιείται συσκευή: NVIDIA CUDA")
else:
    device = torch.device("cpu")
    print("💻 Χρησιμοποιείται συσκευή: CPU")


🔥 Χρησιμοποιείται συσκευή: Apple Silicon GPU (MPS)


## 📚 Εισαγωγή Βιβλιοθηκών
Οι κύριες βιβλιοθήκες που χρησιμοποιήθηκαν είναι:

- torch, torchtext: για την κατασκευή και εκπαίδευση του νευρωνικού μοντέλου.
- datasets: για εύκολη φόρτωση του IMDB dataset και άλλων πηγών.
- numpy, matplotlib, collections, re: για υποστήριξη, οπτικοποίηση και προεπεξεργασία.

In [ ]:
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset

from datasets import load_dataset
import numpy as np
from collections import Counter
import re
import random


## 📥 Φόρτωση Δεδομένων – AG News Dataset

Το AG News dataset που χρησιμοποιήθηκε προέρχεται από την πηγή:
[Kaggle - AG News Classification Dataset](https://www.kaggle.com/datasets/amananandrai/ag-news-classification-dataset)

Αποτελείται από δύο αρχεία `.csv`:
- `train.csv` με 120.000 δείγματα
- `test.csv` με 7.600 δείγματα

Κάθε εγγραφή περιλαμβάνει:
- Κατηγορία (`Class Index`)
- Τίτλο της είδησης (`Title`)
- Σύντομη περιγραφή (`Description`)

Οι τίτλοι και περιγραφές συγχωνεύονται σε ένα ενιαίο string ώστε να χρησιμοποιηθούν ως είσοδοι στο μοντέλο.


In [194]:
import pandas as pd

# Φόρτωση των CSV αρχείων
train_data = pd.read_csv('train.csv')
test_data = pd.read_csv('test.csv')

# Δημιουργία datasets (label, κείμενο) 
train_dataset = [(label, train_data['Title'][i] + ' ' + train_data['Description'][i]) for i, label in enumerate(train_data['Class Index'])]
test_dataset = [(label, test_data['Title'][i] + ' ' + test_data['Description'][i]) for i, label in enumerate(test_data['Class Index'])]

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")


Training samples: 120000
Test samples: 7600


## ✂️ Tokenization – Basic English Tokenizer

Για την μετατροπή του κειμένου σε tokens χρησιμοποιήθηκε ο ενσωματωμένος tokenizer της βιβλιοθήκης torchtext, συγκεκριμένα η μέθοδος basic_english.

Αυτός ο tokenizer:

- Μετατρέπει όλα τα γράμματα σε πεζά
- Αφαιρεί σημεία στίξης
- Διαχωρίζει λέξεις με βάση τα κενά

In [160]:
#device = torch.device("cpu")

from torchtext.data import get_tokenizer

# Basic English tokenizer (lowercase, αφαιρεί σημεία στίξης κλπ)
tokenizer = get_tokenizer("basic_english")



## 📘 Δημιουργία Λεξιλογίου (Vocabulary)

Σύμφωνα με τις οδηγίες της εκφώνησης, δημιουργείται ένα λεξιλόγιο (`vocab`) που περιλαμβάνει:

- Όλες τις λέξεις που εμφανίζονται τουλάχιστον **10 φορές** στα training και test δεδομένα.
- Τα ειδικά tokens:
  - `<PAD>` για padding ακολουθιών
  - `<UNK>` για άγνωστες λέξεις (out-of-vocabulary)

Η διαδικασία περιλαμβάνει χρήση της συνάρτησης `build_vocab_from_iterator` από τη βιβλιοθήκη `torchtext`.


In [161]:
from torchtext.vocab import build_vocab_from_iterator

# Συνάρτηση που παράγει λίστα tokenized λέξεων
def build_vocabulary(datasets):
    for dataset in datasets:
        for _, text in dataset:
            yield tokenizer(text)

# Δημιουργία λεξιλογίου από train + test dataset
vocab = build_vocab_from_iterator(
    build_vocabulary([train_dataset, test_dataset]), 
    min_freq=10,
    specials=["<PAD>", "<UNK>"]
)

# Ορισμός default index σε <UNK> (unknown words)
vocab.set_default_index(vocab["<UNK>"])

print(f"Vocabulary size: {len(vocab)}")


Vocabulary size: 21254


## ⚙️ Δημιουργία Συνάρτησης `collate_batch`

Για την εκπαίδευση του μοντέλου με `DataLoader`, απαιτείται μια custom `collate_fn` συνάρτηση που μετατρέπει κάθε batch από raw κείμενα και labels σε έτοιμα tensors για το μοντέλο.

Η παρακάτω συνάρτηση `collate_batch()` υλοποιεί:

- Tokenization των κειμένων
- Μετατροπή σε indices μέσω του λεξιλογίου (`vocab`)
- Truncation ή Padding των ακολουθιών σε σταθερό μήκος
- Optionally: αφαίρεση -1 από labels, αν ξεκινούν από 1 (π.χ. AGNews)

In [162]:
import torch

MAX_WORDS = 25

def collate_batch(batch, shift_labels=True):
    Y, X = list(zip(*batch))
    Y = torch.tensor(Y)
    
    if shift_labels:
        Y = Y - 1  # αλλαγή για datasets όπως AGNews
    else: 
        Y
        
    
    # Tokenization + μετατροπή σε indices
    X = [vocab(tokenizer(text)) for text in X]
    
    # Truncation/Padding σε μήκος MAX_WORDS
    X = [
        tokens + [vocab['<PAD>']] * (MAX_WORDS - len(tokens)) if len(tokens) < MAX_WORDS else tokens[:MAX_WORDS]
        for tokens in X
    ]
    
    return torch.tensor(X, dtype=torch.int64).to(device), Y.to(device)


## 📦 Δημιουργία DataLoaders

Για τη διαδικασία εκπαίδευσης και αξιολόγησης χρησιμοποιούνται αντικείμενα `DataLoader`, τα οποία αναλαμβάνουν:

- Τη δημιουργία batches στα δεδομένα
- Τη χρήση της custom συνάρτησης `collate_batch` για tokenization και padding
- Shuffle των δεδομένων μόνο στο training set
- Επιτάχυνση μέσω lazy-loading, batch-wise μεταφορά σε GPU/MPS

Η τιμή του `BATCH_SIZE` ορίστηκε σε 1024 για ταχύτητα και επαρκή στατιστική ποικιλία σε κάθε batch.


In [163]:
from torch.utils.data import DataLoader

# HYPER-PARAMETERS
BATCH_SIZE = 1024

# DataLoader για training set
train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    collate_fn=collate_batch
)

# DataLoader για test set
test_loader = DataLoader(
    test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    collate_fn=collate_batch
)


## 🧠 Ορισμός Μοντέλου – `RNNClassifier` (AGNews & IMDB)

Η κλάση `RNNClassifier` αποτελεί τον πυρήνα του μοντέλου και έχει σχεδιαστεί με στόχο:

- Την υποστήριξη διαφορετικών τύπων RNN (`RNN`, `LSTM`)
- Την επιλογή αριθμού στρωμάτων και κατεύθυνσης (μονοκατευθυντικό ή bidirectional)
- Τη δυνατότητα εισαγωγής προεκπαιδευμένων embeddings
- Την επιλογή να "παγώσουν" (freeze) τα embeddings ή να συνεχίσουν να μαθαίνουν


In [164]:
import torch.nn as nn
import torch.nn.functional as F

class RNNClassifier(nn.Module):
    def __init__(self, 
                 vocab_size, 
                 embedding_dim, 
                 hidden_dim, 
                 output_dim,
                 rnn_type="rnn",          # "rnn" ή "lstm"
                 num_layers=1, 
                 bidirectional=False,
                 pretrained_embeddings=None,    # προσθήκη ερωτήματος 4
                 freeze_embeddings=False        # προσθήκη ερωτήματος 5
                 ): 
        
        super(RNNClassifier, self).__init__()
        
        self.embedding = nn.Embedding(vocab_size, embedding_dim)

        if pretrained_embeddings is not None:                           # προσθήκη ερωτήματος 4
            self.embedding.weight.data.copy_(pretrained_embeddings)     # προσθήκη ερωτήματος 4

        if freeze_embeddings:                               # προσθήκη ερωτήματος 5
            self.embedding.weight.requires_grad = False     # προσθήκη ερωτήματος 5

        if rnn_type == "rnn":
            self.rnn = nn.RNN(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=num_layers,
                bidirectional=bidirectional,
                batch_first=True
            )
        elif rnn_type == "lstm":
            self.rnn = nn.LSTM(
                input_size=embedding_dim,
                hidden_size=hidden_dim,
                num_layers=num_layers,
                bidirectional=bidirectional,
                batch_first=True
            )
        else:
            raise ValueError("Unsupported rnn_type. Choose 'rnn' or 'lstm'.")

        direction_factor = 2 if bidirectional else 1
        self.fc = nn.Linear(hidden_dim * direction_factor, output_dim)

    def forward(self, x):
        x = self.embedding(x)
        rnn_out, _ = self.rnn(x)
        out = self.fc(rnn_out[:, -1, :])  # παίρνουμε το τελευταίο output του sequence
        return F.softmax(out, dim=1)
    


## 🧪 Παραδείγματα Μοντέλων

Ακολουθούν παραδείγματα κατασκευής δύο βασικών μοντέλων με χρήση της παραμετρικής κλάσης `RNNClassifier`:

### 🔹 1-layer RNN

Το πιο απλό σενάριο: RNN με μία κρυφή στρώση, μονοκατευθυντικό.  
Χρήσιμο για benchmarking ή εισαγωγικά πειράματα.

### 🔸 2-layer Bidirectional LSTM (BiLSTM)
Ισχυρότερο μοντέλο: LSTM, διπλής κατεύθυνσης (biLSTM), με 2 κρυφές στρώσεις.
Αξιοποιεί καλύτερα τη δομή του κειμένου καθώς διαβάζει και από τις δύο κατευθύνσεις.

In [165]:
# 1-layer RNN
model = RNNClassifier(
    vocab_size=len(vocab),
    embedding_dim=100,
    hidden_dim=64,
    output_dim=4,
    rnn_type="rnn",
    num_layers=1,
    bidirectional=False
).to(device)


# 2-layer BiLSTM
model = RNNClassifier(
    vocab_size=len(vocab),
    embedding_dim=100,
    hidden_dim=64,
    output_dim=4,
    rnn_type="lstm",
    num_layers=2,
    bidirectional=True
).to(device)


## ⚙️ Ορισμός Υπερπαραμέτρων

Οι βασικές υπερπαράμετροι που ορίζουν τη συμπεριφορά και την απόδοση του μοντέλου είναι οι εξής:


In [ ]:
MAX_WORDS = 25         # Μήκος token sequence (μετά από padding/truncation)
EPOCHS = 15            # Πλήρεις επαναλήψεις πάνω στο training set
LEARNING_RATE = 1e-3   # Ρυθμός μάθησης για τον optimizer
BATCH_SIZE = 1024      # Μέγεθος mini-batch (επηρεάζει μνήμη & ταχύτητα)
EMBEDDING_DIM = 100    # Διάσταση των word embeddings
HIDDEN_DIM = 64        # Διάσταση των hidden states του RNN


## 🧮 Συναρτήσεις Απώλειας και Βελτιστοποίησης

Για την εκπαίδευση των μοντέλων χρησιμοποιήθηκαν:

- Η συνάρτηση απώλειας `CrossEntropyLoss`, κατάλληλη για multi-class classification
- Ο βελτιστοποιητής `Adam`, γνωστός για γρήγορη και σταθερή σύγκλιση


In [167]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)


## 📉 Συνάρτηση Αξιολόγησης – `evaluate_model`

Η συνάρτηση `evaluate_model()` αξιολογεί την απόδοση του μοντέλου σε test set (ή validation set), χωρίς να επηρεάζει τα gradients.

Επιστρέφει:

- Τη μέση τιμή απώλειας στο dataset (`total_loss / len(dataloader)`)
- Τα πραγματικά labels (`true_labels`)
- Τις προβλέψεις του μοντέλου (`pred_labels`)

In [168]:
def evaluate_model(model, dataloader, loss_fn):
    model.eval()
    total_loss = 0
    true_labels = []
    pred_labels = []

    with torch.no_grad():
        for X_batch, y_batch in dataloader:
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            total_loss += loss.item()

            preds = outputs.argmax(dim=1)
            true_labels.append(y_batch.cpu())
            pred_labels.append(preds.cpu())

    true_labels = torch.cat(true_labels)
    pred_labels = torch.cat(pred_labels)
    
    return total_loss / len(dataloader), true_labels.numpy(), pred_labels.numpy()


## 🏋️ Συνάρτηση Εκπαίδευσης – `train_model`

Η συνάρτηση `train_model()` υλοποιεί τη βασική ροή εκπαίδευσης ενός νευρωνικού μοντέλου για `epochs` επαναλήψεις.

Για κάθε εποχή πραγματοποιούνται:

- Εκπαίδευση πάνω στο training set
- Αξιολόγηση στο test set
- Εκτύπωση των βασικών μετρικών: train loss, val loss και val accuracy


In [169]:
from tqdm import tqdm
from sklearn.metrics import accuracy_score

def train_model(model, train_loader, test_loader, optimizer, loss_fn, epochs):
    for epoch in range(1, epochs + 1):
        model.train()
        train_losses = []

        print(f"Epoch {epoch}")
        
        for X_batch, y_batch in tqdm(train_loader):
            optimizer.zero_grad()
            outputs = model(X_batch)
            loss = loss_fn(outputs, y_batch)
            loss.backward()
            optimizer.step()

            train_losses.append(loss.item())
        
        avg_train_loss = sum(train_losses) / len(train_losses)
        
        # Evaluation on test set
        val_loss, y_true, y_pred = evaluate_model(model, test_loader, loss_fn)
        val_acc = accuracy_score(y_true, y_pred)

        print(f"Train Loss: {avg_train_loss:.4f} | Val Loss: {val_loss:.4f} | Val Accuracy: {val_acc:.4f}\n")


## 🚀 Εκπαίδευση RNN Μοντέλου (1-Layer RNN)

Παρακάτω παρουσιάζεται ένα πλήρες παράδειγμα εκπαίδευσης ενός απλού 1-layer RNN μοντέλου πάνω στο AGNews dataset:

- RNN (όχι LSTM)
- 1 στρώση (layer)
- Μονοκατευθυντικό (όχι bidirectional)
- 4 κατηγορίες εξόδου (World, Sports, Business, Sci/Tech)

In [ ]:
# Δημιουργία 1-layer RNN
model = RNNClassifier(
    vocab_size=len(vocab),
    embedding_dim=EMBEDDING_DIM,
    hidden_dim=HIDDEN_DIM,
    output_dim=4,            # 4 κατηγορίες (World, Sports, Business, Sci/Tech)
    rnn_type="rnn",          # απλό RNN
    num_layers=1,            # 1 layer
    bidirectional=False      # όχι bidirectional
).to(device)

# Loss και Optimizer
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)

train_model(
    model=model,
    train_loader=train_loader,
    test_loader=test_loader,
    optimizer=optimizer,
    loss_fn=loss_fn,
    epochs=EPOCHS
)

## 🧪 Ορισμός Μοντέλων για Πειραματισμό

Όπως ζητείται στο Ερώτημα 1, εξετάζονται οι εξής έξι παραλλαγές RNN-based μοντέλων:

- Τύπος RNN: `RNN` ή `LSTM`
- Μονοκατευθυντικά ή διπλής κατεύθυνσης (bidirectional)
- Ένα ή δύο επίπεδα (layers)

Ο παρακάτω πίνακας περιέχει τα configurations όλων των μοντέλων:

In [173]:
model_configs = [
    {"name": "1RNN",       "rnn_type": "rnn",  "num_layers": 1, "bidirectional": False},
    {"name": "1Bi-RNN",    "rnn_type": "rnn",  "num_layers": 1, "bidirectional": True},
    {"name": "2Bi-RNN",    "rnn_type": "rnn",  "num_layers": 2, "bidirectional": True},
    {"name": "1LSTM",      "rnn_type": "lstm", "num_layers": 1, "bidirectional": False},
    {"name": "1Bi-LSTM",   "rnn_type": "lstm", "num_layers": 1, "bidirectional": True},
    {"name": "2Bi-LSTM",   "rnn_type": "lstm", "num_layers": 2, "bidirectional": True}
]


## 🔁 Συνάρτηση Εκπαίδευσης για Κάθε Μοντέλο – `run_single_model()`

Η συνάρτηση `run_single_model()` αναλαμβάνει:

- Να χτίσει το μοντέλο με βάση ένα configuration (`config`)
- Να το εκπαιδεύσει για `EPOCHS` επαναλήψεις
- Να το αξιολογήσει
- Να επιστρέψει τα βασικά στατιστικά για πίνακες σύγκρισης και error analysis

### 🔧 Είσοδοι (Arguments):

| Παράμετρος              | Περιγραφή                                               |
|-------------------------|----------------------------------------------------------|
| `config`                | Λεξικό με στοιχεία αρχιτεκτονικής (όνομα, layers κ.λπ.) |
| `pretrained_embeddings` | Embedding matrix (GloVe κ.λπ.) ή `None`                 |
| `freeze_embeddings`     | Αν θα εκπαιδεύονται τα embeddings ή όχι                |
| `vocab_to_use`          | Το λεξιλόγιο που θα χρησιμοποιηθεί                     |
| `output_dim`            | Αριθμός εξόδων (4 για AGNews, 2 για IMDB)              |
| `train_loader`          | DataLoader για training set                            |
| `test_loader`           | DataLoader για test set     

In [174]:
import time

def run_single_model(config, 
                     pretrained_embeddings=None,  # προσθήκη ερωτήματος 4 
                     freeze_embeddings=False,  # προσθήκη ερωτήματος 5
                     vocab_to_use=vocab,  # προσθήκη ερωτήματος 6
                     output_dim=4,              # προσθήκη ερωτήματος 6
                     train_loader=train_loader,         # προσθήκη ερωτήματος 6
                     test_loader=test_loader           # προσθήκη ερωτήματος 6
                     ):    
    print(f"\n🔄 Training: {config['name']}")
    
        

    model = RNNClassifier(
        vocab_size=len(vocab_to_use),
        embedding_dim=EMBEDDING_DIM,
        hidden_dim=HIDDEN_DIM,
        output_dim=output_dim,
        rnn_type=config["rnn_type"],
        num_layers=config["num_layers"],
        bidirectional=config["bidirectional"],
        pretrained_embeddings=pretrained_embeddings,  # προσθήκη ερωτήματος 4
        freeze_embeddings=freeze_embeddings # προσθήκη ερωτήματος 5
    ).to(device)
    
    optimizer = torch.optim.Adam(model.parameters(), lr=LEARNING_RATE)
    loss_fn = nn.CrossEntropyLoss()

    start = time.time()
    train_model(model, train_loader, test_loader, optimizer, loss_fn, epochs=EPOCHS)
    end = time.time()
    
    total_params = sum(p.numel() for p in model.parameters() if p.requires_grad)
    val_loss, y_true, y_pred = evaluate_model(model, test_loader, loss_fn)
    val_acc = accuracy_score(y_true, y_pred)

    return {
        "Model": config['name'],
        "Accuracy": round(val_acc * 100, 2),
        "Parameters": total_params,
        "Time (sec)": round((end - start) / EPOCHS, 2),
        "y_pred": y_pred   # <-- Προσθέτουμε τα predictions
    }

## 🧪 Εκτέλεση Όλων των Μοντέλων – `run_experiments()`

Η συνάρτηση `run_experiments()` χρησιμοποιείται για να εκτελέσει **όλα τα μοντέλα που καθορίζονται στον πίνακα `model_configs`**, και να επιστρέψει:

- Ένα DataFrame με μετρικές απόδοσης για κάθε μοντέλο
- Ένα dictionary με τα predicted labels (`y_pred`) κάθε μοντέλου (για ανάλυση σφαλμάτων)


In [175]:
import pandas as pd

def run_experiments(configs, pretrained_embeddings=None, freeze_embeddings=False, vocab_to_use=vocab, output_dim=4, train_loader=train_loader, test_loader=test_loader): # pretrained_embeddings προσθήκη ερωτήματος 4 | freeze_embeddings προσθήκη ερωτήματος 5
    results = []
    preds_dict = {}

    for cfg in configs:
        result = run_single_model(cfg, pretrained_embeddings=pretrained_embeddings, freeze_embeddings=freeze_embeddings,
        vocab_to_use=vocab_to_use,
        output_dim=output_dim,
        train_loader=train_loader,
        test_loader=test_loader
            ) # pretrained_embeddings προσθήκη ερωτήματος 4 | freeze_embeddings προσθήκη ερωτήματος 5
        preds_dict[cfg['name']] = result.pop('y_pred')  # Αποθηκεύουμε y_preds ξεχωριστά
        results.append(result)

    df_results = pd.DataFrame(results)
    return df_results, preds_dict


# Απαντήσεις Ερωτημάτων

## 📊 Ερώτημα Γ.1  – Εκτέλεση Πειραμάτων & Συμπλήρωση Πίνακα

Σε αυτό το ερώτημα εξετάζονται έξι διαφορετικά μοντέλα που διαφέρουν ως προς:

- Τον τύπο RNN (`RNN` ή `LSTM`)
- Το αν είναι μονοκατευθυντικά ή bidirectional
- Τον αριθμό των επιπέδων (layers): 1 ή 2

In [176]:
results_table, preds_dict = run_experiments(model_configs)


🔄 Training: 1RNN
Epoch 1


100%|██████████| 118/118 [00:03<00:00, 36.01it/s]


Train Loss: 1.2895 | Val Loss: 1.1397 | Val Accuracy: 0.6053

Epoch 2


100%|██████████| 118/118 [00:03<00:00, 37.01it/s]


Train Loss: 1.0579 | Val Loss: 1.0076 | Val Accuracy: 0.7370

Epoch 3


...

Train Loss: 0.8107 | Val Loss: 0.8607 | Val Accuracy: 0.8804

Epoch 14


100%|██████████| 118/118 [00:06<00:00, 18.06it/s]


Train Loss: 0.8101 | Val Loss: 0.8571 | Val Accuracy: 0.8838

Epoch 15


100%|██████████| 118/118 [00:06<00:00, 18.29it/s]


Train Loss: 0.8068 | Val Loss: 0.8579 | Val Accuracy: 0.8841



### Κατασκευή Πίνακα Ερωτήματος Γ.1 

In [179]:
# Transpose results to match the table format in the assignment
df_transposed = results_table.set_index("Model").T

# Round accuracy to 2 decimals, format numbers
df_display = pd.DataFrame({
    col: [
        f"{df_transposed.loc['Accuracy', col]:.2f}",
        f"{int(df_transposed.loc['Parameters', col]):,}",
        f"{df_transposed.loc['Time (sec)', col]:.2f}"
    ]
    for col in df_transposed.columns
}, index=["Accuracy", "Parameters", "Time cost"])

df_display

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,86.64,86.80,85.67,88.59,88.12,88.41
Parameters,"2,136,284","2,147,164","2,171,996","2,168,156","2,210,908","2,310,236"
Time cost,3.42,4.47,6.84,4.27,4.98,8.32


### Ερώτημα Γ.1 – Συμπεράσματα

Από τη συγκριτική μελέτη των έξι μοντέλων (RNN & LSTM, με/χωρίς bidirectionality και 1 ή 2 στρώματα) προκύπτουν τα εξής γενικά συμπεράσματα:

- **Η αύξηση της ακρίβειας συνοδεύεται σταθερά από αύξηση του χρόνου εκπαίδευσης και της πολυπλοκότητας του μοντέλου** (παραμέτρων). Το μοντέλο `2Bi-LSTM`, για παράδειγμα, είχε το μεγαλύτερο accuracy αλλά και το υψηλότερο κόστος χρόνου και αριθμό παραμέτρων.

- **Η τελική ακρίβεια όλων των μοντέλων ήταν πολύ κοντά** (από 85.88% έως 88.59%). Αυτό σημαίνει πως δεν είναι εύκολο να εξαχθούν απόλυτα συμπεράσματα υπέρ κάποιας αρχιτεκτονικής.

- **Η χρήση bidirectional ροής και δεύτερου στρώματος δεν φαίνεται να προσφέρει σταθερά πλεονεκτήματα ως προς την ακρίβεια**. Ορισμένα Bi-models είχαν ελαφρώς χαμηλότερη ακρίβεια από τα απλά.

📌 Συμπερασματικά, η επιλογή του κατάλληλου μοντέλου εξαρτάται κυρίως από **τις ανάγκες του συστήματος**: αν προέχει η ακρίβεια ή η ταχύτητα/απλότητα.


## 🧩 Ερώτημα Γ.2 – Εντοπισμός Κοινών Λανθασμένων Προβλέψεων

Στόχος του ερωτήματος είναι να εντοπίσουμε περιπτώσεις όπου **όλα τα μοντέλα** αποτυγχάνουν να προβλέψουν σωστά την κατηγορία ενός κειμένου.

### Δημιουργία DataFrame με Όλες τις Προβλέψεις

Για κάθε sample του test set, δημιουργούμε ένα νέο `DataFrame` που περιλαμβάνει:

- Το πλήρες κείμενο (Title + Description)
- Το ground truth label (0–3)
- Το κατηγορικό όνομα της κατηγορίας (`World`, `Sports`, κ.λπ.)
- Τις προβλέψεις κάθε μοντέλου (`1RNN`, `1Bi-RNN`, ...)

In [180]:
# Ground truth labels (0–3)
y_true = test_data["Class Index"].values - 1

# Map index → category
label_names = ["World", "Sports", "Business", "Sci/Tech"]

# Δημιουργία βασικού DataFrame
df_errors = pd.DataFrame({
    "text": test_data['Title'] + ' ' + test_data['Description'],
    "true_label": y_true
})

# Προσθήκη προβλέψεων από όλα τα μοντέλα
for model_name, preds in preds_dict.items():
    df_errors[model_name] = preds

# Αντιστοιχία σε κατηγορία (text)
df_errors["true_category"] = df_errors["true_label"].apply(lambda i: label_names[i])
for model_name in preds_dict:
    df_errors[model_name] = df_errors[model_name].apply(lambda i: label_names[i])


### ❌ Εντοπισμός Κοινών Λαθών
Ορίζεται ως κοινό λάθος όταν όλα τα μοντέλα διαφωνούν με την πραγματική κατηγορία:

In [182]:
from collections import Counter


def majority_vote(row):
    preds = [row[m] for m in preds_dict]
    return Counter(preds).most_common(1)[0][0]


# Βρίσκουμε πού όλα τα μοντέλα κάνουν λάθος
all_wrong_mask = df_errors.apply(
    lambda row: all(row[m] != row["true_category"] for m in preds_dict), 
    axis=1
)

# Φιλτράρουμε τα κοινά λάθη
df_all_wrong = df_errors[all_wrong_mask].copy()

# Υπολογίζουμε majority vote για κάθε κοινό λάθος
df_all_wrong["majority_vote"] = df_all_wrong.apply(majority_vote, axis=1)

print(f" * Συνολικά κοινά λάθη από όλα τα μοντέλα: {len(df_all_wrong)}")
print("\n * Κατανομή κοινών λαθών ανά κατηγορία:")
print(df_all_wrong["true_category"].value_counts())

 * Συνολικά κοινά λάθη από όλα τα μοντέλα: 333

 * Κατανομή κοινών λαθών ανά κατηγορία:
true_category
Business    125
World       116
Sci/Tech     77
Sports       15
Name: count, dtype: int64


In [183]:
# Εμφάνιση πρώτων 10 κοινών λαθών με τις προβλέψεις
df_all_wrong[["text", "true_category", *preds_dict.keys(), "majority_vote"]].head(10)

,text,true_category,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM,majority_vote
79,Live: Olympics day four Richard Faulds and Ste...,World,Sports,Sports,Sports,Sports,Sports,Sports,Sports
83,Intel to delay product aimed for high-definiti...,Business,Sci/Tech,Sci/Tech,Sci/Tech,Sci/Tech,Sci/Tech,Sci/Tech,Sci/Tech
88,"U.S. Misses Cut in Olympic 100 Free ATHENS, Gr...",World,Sports,Sports,Sports,Sports,Sports,Sports,Sports
89,Consumers Would Pay In Phone Proposal A propos...,Sci/Tech,Business,Business,Business,Business,Business,Business,Business
106,Stocks Climb on Drop in Consumer Prices NEW YO...,World,Business,Business,Business,Business,Business,Business,Business
110,Yahoo! Ups Ante for Small Businesses Web giant...,Business,Sci/Tech,Sci/Tech,Sci/Tech,Sci/Tech,Sci/Tech,Sci/Tech,Sci/Tech
120,Oil prices bubble to record high The price of ...,World,Business,Business,Business,Business,Business,Business,Business
154,"Google Lowers Its IPO Price Range SAN JOSE, Ca...",World,Sci/Tech,Business,Sci/Tech,Business,Business,Sci/Tech,Sci/Tech
196,Stock Prices Climb Ahead of Google IPO NEW YOR...,World,Business,Business,Business,Business,Business,Business,Business
200,Strong Family Equals Strong Education Single m...,Sci/Tech,Business,World,Business,World,Business,World,Business


### Ερώτημα Γ.2 – Συμπεράσματα

Αναλύοντας τα δείγματα του test set στα οποία **όλα τα RNN-based μοντέλα έκαναν λάθος**, προκύπτουν τα εξής:

- ❌ **Συνολικά 333 περιπτώσεις** προβλέφθηκαν λανθασμένα από όλα τα μοντέλα.

- 📊 Η **κατανομή των κοινών λαθών** ανά κατηγορία έχει ιδιαίτερο ενδιαφέρον:
  - `World`: 125 λάθη
  - `Business`: 116 λάθη
  - `Sci/Tech`: 77 λάθη
  - `Sports`: **μόνο 15 λάθη**

- ⚽ Η κατηγορία `Sports` έχει **πολύ λιγότερα κοινά λάθη**, κάτι που υποδηλώνει ότι τα μοντέλα αναγνωρίζουν πιο εύκολα τα αθλητικά κείμενα.

- Στις περισσότερες περιπτώσεις που παρατηρήθηκε λάθος:
  - **Στη πλειοψηφία τους τα μοντέλα συμφώνησαν** στο λάθος (π.χ. προέβλεψαν "Business" αντί "World").
  - Η **πλειοψηφία των μοντέλων (majority vote)** απέδωσε το ίδιο λάθος label, υποδηλώνοντας κοινή αστοχία κατανόησης.

- Συγκριτικά με τα μοντέλα του Β' μέρους (Naive Bayes & SVM), παρατηρούμε:
  - Παρόμοια κατανομή λαθών.
  - Αντίστοιχα χαρακτηριστικά δυσκολίας στις κατηγορίες `World` και `Business`.

📌 Γενικό συμπέρασμα:
> Παρότι τα RNN-based μοντέλα είναι πιο εξελιγμένα, αποτυγχάνουν σε **κείμενα με ασαφή ή επικαλυπτόμενα συμφραζόμενα**, και σε αρκετές περιπτώσεις η λανθασμένη πρόβλεψη είναι κοινή για όλα.


## 🔠 Ερώτημα Γ.3 – Μεταβολή της παραμέτρου `MAX_WORDS`

Σε αυτό το ερώτημα μεταβάλλεται η παράμετρος `MAX_WORDS` από **25 σε 50**, ώστε να επιτρέψουμε μεγαλύτερη πληροφορία ανά δείγμα κειμένου.

---

### 🎯 Σκοπός

Να διερευνηθεί πώς επηρεάζεται:

- Η **απόδοση των μοντέλων** (accuracy)
- Ο **χρόνος εκπαίδευσης**
- Η **πολυπλοκότητα του input** (καθώς τα sequences μεγαλώνουν)


In [184]:

MAX_WORDS = 50

results_50, preds_50 = run_experiments(model_configs)



🔄 Training: 1RNN
Epoch 1


100%|██████████| 118/118 [00:05<00:00, 23.46it/s]


Train Loss: 1.3765 | Val Loss: 1.3508 | Val Accuracy: 0.3312

Epoch 2


100%|██████████| 118/118 [00:04<00:00, 24.45it/s]


Train Loss: 1.3350 | Val Loss: 1.3147 | Val Accuracy: 0.3858

Epoch 3


...

Train Loss: 0.8401 | Val Loss: 0.8577 | Val Accuracy: 0.8821

Epoch 14


100%|██████████| 118/118 [00:14<00:00,  8.24it/s]


Train Loss: 0.8360 | Val Loss: 0.8612 | Val Accuracy: 0.8796

Epoch 15


100%|██████████| 118/118 [00:19<00:00,  6.03it/s]


Train Loss: 0.8347 | Val Loss: 0.8627 | Val Accuracy: 0.8779



In [185]:
# Transpose results to match the table format in the assignment
df_transposed_50 = results_50.set_index("Model").T

# Round accuracy to 2 decimals, format numbers
df_display_50 = pd.DataFrame({
    col: [
        f"{df_transposed_50.loc['Accuracy', col]:.2f}",
        f"{int(df_transposed_50.loc['Parameters', col]):,}",
        f"{df_transposed_50.loc['Time (sec)', col]:.2f}"
    ]
    for col in df_transposed_50.columns
}, index=["Accuracy", "Parameters", "Time cost"])

df_display_50

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,42.03,45.03,44.87,88.01,87.30,87.79
Parameters,"2,136,284","2,147,164","2,171,996","2,168,156","2,210,908","2,310,236"
Time cost,5.10,7.88,12.25,5.54,10.44,16.42


### Ερώτημα Γ.3 – Συμπεράσματα

Αλλάξαμε την παράμετρο `MAX_WORDS` από 25 σε 50 και επανεκπαιδεύσαμε όλα τα μοντέλα.

Τα αποτελέσματα έδειξαν:

- **Τα accuracies των απλών RNN (1RNN, 1Bi-RNN, 2Bi-RNN) έπεσαν κατακόρυφα**, σχεδόν στο μισό ή και πολύ περισσότερο κατα περίπτωση!
  - Π.χ. το 1RNN έπεσε από ~87% σε μόλις ~42%.
- **Τα LSTM-based μοντέλα (1LSTM, 1Bi-LSTM, 2Bi-LSTM) παρέμειναν σταθερά σε υψηλό επίπεδο** (περίπου 87–88%).

-  **Ο χρόνος εκπαίδευσης αυξήθηκε σημαντικά**:
  - Π.χ. το 2Bi-LSTM ανέβηκε από ~8 sec σε ~16 sec ανά epoch.

- **Ο αριθμός παραμέτρων παρέμεινε ίδιος** σε όλα τα μοντέλα:
  - Αυτό ήταν αναμενόμενο, αφού ο αριθμός παραμέτρων εξαρτάται από:
    - Το μέγεθος του λεξιλογίου
    - Το μέγεθος των embeddings
    - Το μέγεθος του hidden layer
  - Και **όχι** από το μήκος των εισόδων (`MAX_WORDS`).

---

📌 Γενικό Συμπέρασμα:

- Η αύξηση του μήκους των κειμένων σε 50 tokens:
  - Δυσκόλεψε τα απλά RNN μοντέλα (πιθανόν λόγω προβλημάτων "vanishing gradient" και αδυναμίας να διαχειριστούν μεγαλύτερες ακολουθίες).
  - Δεν επηρέασε σημαντικά τα LSTM μοντέλα, τα οποία είναι σχεδιασμένα για μακρύτερες ακολουθίες.
- Υπάρχει ξεκάθαρη επιβεβαίωση ότι **τα LSTM μοντέλα είναι πιο ανθεκτικά σε μεγαλύτερα κείμενα** και **διαχειρίζονται καλύτερα το context**.



## 🧠 Ερώτημα Γ.4 – Εκπαίδευση με Προεκπαιδευμένα GloVe Embeddings

Σε αυτό το ερώτημα γίνεται χρήση **προεκπαιδευμένων διανυσμάτων λέξεων (GloVe)** στη θέση των randomly initialized embeddings που χρησιμοποιήθηκαν στα προηγούμενα ερωτήματα.


### Φόρτωση GloVe Embeddings
Τα GloVe vectors φορτώνονται από το αρχείο `glove.6B.100d.txt`, το οποίο περιέχει 400.000 λέξεις με διαστάσεις 100.

In [186]:
# Διαβάζουμε το αρχείο
def load_glove_embeddings(glove_path):
    embeddings_index = {}
    with open(glove_path, encoding='utf8') as f:
        for line in f:
            values = line.strip().split()
            word = values[0]
            vector = torch.tensor([float(x) for x in values[1:]], dtype=torch.float32)
            embeddings_index[word] = vector
    return embeddings_index

# Παράδειγμα
glove_embeddings = load_glove_embeddings('glove.6B.100d.txt')
print(f"Loaded {len(glove_embeddings)} word vectors from GloVe.")


Loaded 400000 word vectors from GloVe.


### Δημιουργία Pretrained Embedding Matrix
Για να εισαχθούν τα vectors στο μοντέλο, φτιάχνεται μια embedding matrix:

In [187]:
def create_embedding_matrix(vocab, glove_embeddings, embedding_dim):
    matrix = torch.zeros(len(vocab), embedding_dim)
    for word, idx in vocab.get_stoi().items():
        if word in glove_embeddings:
            matrix[idx] = glove_embeddings[word]
        else:
            matrix[idx] = torch.randn(embedding_dim) * 0.01  # μικρό random init για άγνωστες λέξεις
    return matrix


In [188]:
embedding_matrix = create_embedding_matrix(vocab, glove_embeddings, EMBEDDING_DIM)

### 🛠️ Σημείωση για την Υλοποίηση – Τεχνικές Προσαρμογές Ερωτήματος 4

Για να υποστηριχθεί η χρήση προεκπαιδευμένων GloVe embeddings, έγιναν οι παρακάτω τροποποιήσεις στον υπάρχοντα κώδικα:

- Οι αλλαγές εντοπίζονται **μέσα στις συναρτήσεις και τις κλάσεις** με **σαφή inline σχολιασμό** της μορφής:
  
  ```python
  # προσθήκη ερωτήματος 4

### Εκτέλεση Πειραμάτων με GloVe Embeddings
Τα μοντέλα τρέχουν με το embedding_matrix ως αρχικοποίηση

In [ ]:
MAX_WORDS = 25

# Εκπαίδευση με προ-εκπαιδευμένα GloVe embeddings
results_glove, preds_glove = run_experiments(model_configs, pretrained_embeddings=embedding_matrix)



🔄 Training: 1RNN
Epoch 1


100%|██████████| 118/118 [00:03<00:00, 30.60it/s]


Train Loss: 1.0526 | Val Loss: 0.9051 | Val Accuracy: 0.8411

Epoch 2


100%|██████████| 118/118 [00:03<00:00, 31.82it/s]


Train Loss: 0.8857 | Val Loss: 0.8724 | Val Accuracy: 0.8705

Epoch 3


...

Train Loss: 0.8037 | Val Loss: 0.8347 | Val Accuracy: 0.9054

Epoch 14


100%|██████████| 118/118 [00:09<00:00, 12.65it/s]


Train Loss: 0.8017 | Val Loss: 0.8326 | Val Accuracy: 0.9097

Epoch 15


100%|██████████| 118/118 [00:08<00:00, 13.68it/s]


Train Loss: 0.8006 | Val Loss: 0.8333 | Val Accuracy: 0.9079



In [190]:
# Transpose results to match the table format in the assignment
df_transposed_glove_25 = results_glove.set_index("Model").T

# Round accuracy to 2 decimals, format numbers
df_display_glove_25 = pd.DataFrame({
    col: [
        f"{df_transposed_glove_25.loc['Accuracy', col]:.2f}",
        f"{int(df_transposed_glove_25.loc['Parameters', col]):,}",
        f"{df_transposed_glove_25.loc['Time (sec)', col]:.2f}"
    ]
    for col in df_transposed_glove_25.columns
}, index=["Accuracy", "Parameters", "Time cost"])

df_display_glove_25

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,86.33,88.50,89.49,90.59,90.91,90.79
Parameters,"2,136,284","2,147,164","2,171,996","2,168,156","2,210,908","2,310,236"
Time cost,4.02,4.80,7.86,4.24,5.03,9.56


### Ερώτημα Γ.4 – Χρήση Προεκπαιδευμένων Word Embeddings (GloVe) με MAX_WORDS = 25

Μετά την αλλαγή του `MAX_WORDS` πίσω σε 25 (όπως ζητούσε η εκφώνηση) και την εκπαίδευση των μοντέλων με προ-εκπαιδευμένα embeddings (GloVe 6B, 100d), προκύπτουν τα εξής:

- **Βελτίωση της ακρίβειας** σε όλα τα μοντέλα, περίπου κατά **1–2%** σε σχέση με τα randomly initialized embeddings.

-  **Μικρή μείωση του χρόνου εκπαίδευσης** σε όλα τα μοντέλα (~0.5–1 sec ανά epoch).
  - Πιθανότατα λόγω καλύτερης αρχικοποίησης, που επιτρέπει ταχύτερη σύγκλιση.

- **Καμία διαφορά στον αριθμό παραμέτρων**.
  - Το μέγεθος του μοντέλου εξαρτάται από το μέγεθος του λεξιλογίου και των κρυφών επιπέδων, όχι από το είδος της αρχικοποίησης των embeddings.

---

📌 Γενικό συμπέρασμα:
> Η χρήση προεκπαιδευμένων GloVe embeddings οδηγεί σε καλύτερη αρχική αναπαράσταση των λέξεων, βελτιώνοντας ελαφρώς την τελική ακρίβεια των μοντέλων χωρίς σημαντικό computational overhead.


## 🧊 Ερώτημα Γ.5 – Χρήση Προεκπαιδευμένων GloVe Embeddings με `freeze=True`

Σε αυτό το ερώτημα δοκιμάζουμε το πείραμα του Ερωτήματος 4 **με παγωμένα (frozen) embeddings**, ώστε να **μην ενημερώνονται τα διανύσματα** κατά τη διάρκεια της εκπαίδευσης.

### 🧪 Εκτέλεση Πειράματος

Η μόνη αλλαγή στην κλήση της συνάρτησης `run_experiments()` είναι η προσθήκη του ορίσματος `freeze_embeddings=True`:


In [79]:
results_glove_frozen, preds_glove_frozen = run_experiments(model_configs, pretrained_embeddings=embedding_matrix, freeze_embeddings=True)


🔄 Training: 1RNN
Epoch 1


100%|██████████| 118/118 [00:05<00:00, 22.18it/s]


Train Loss: 1.0680 | Val Loss: 0.9012 | Val Accuracy: 0.8471

Epoch 2


100%|██████████| 118/118 [00:03<00:00, 35.35it/s]


Train Loss: 0.9006 | Val Loss: 0.9069 | Val Accuracy: 0.8354

Epoch 3


...

Train Loss: 0.8373 | Val Loss: 0.8437 | Val Accuracy: 0.8972

Epoch 14


100%|██████████| 118/118 [00:26<00:00,  4.49it/s]


Train Loss: 0.8355 | Val Loss: 0.8428 | Val Accuracy: 0.8987

Epoch 15


100%|██████████| 118/118 [00:27<00:00,  4.32it/s]


Train Loss: 0.8337 | Val Loss: 0.8474 | Val Accuracy: 0.8934



In [80]:
# Transpose results to match the table format in the assignment
df_transposed_glove_25_freeze = results_glove_frozen.set_index("Model").T

# Round accuracy to 2 decimals, format numbers
df_display_glove_25_freeze = pd.DataFrame({
    col: [
        f"{df_transposed_glove_25_freeze.loc['Accuracy', col]:.2f}",
        f"{int(df_transposed_glove_25_freeze.loc['Parameters', col]):,}",
        f"{df_transposed_glove_25_freeze.loc['Time (sec)', col]:.2f}"
    ]
    for col in df_transposed_glove_25_freeze.columns
}, index=["Accuracy", "Parameters", "Time cost"])

df_display_glove_25_freeze

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,86.45,85.18,84.92,89.76,89.83,89.34
Parameters,"10,884","21,764","46,596","42,756","85,508","184,836"
Time cost,3.72,5.58,9.10,6.96,13.03,27.73


### Ερώτημα Γ.5 – Συμπεράσματα

Στο πέμπτο ερώτημα, χρησιμοποιήθηκαν και πάλι τα GloVe προεκπαιδευμένα embeddings (`glove.6B.100d`), αλλά αυτή τη φορά το embedding layer παρέμεινε **"frozen"** (χωρίς ενημέρωση κατά τη διάρκεια της εκπαίδευσης).

Τα βασικά συμπεράσματα είναι:

- **Μικρή μείωση της ακρίβειας** σε όλα τα μοντέλα (~1%) σε σχέση με το σενάριο όπου τα embeddings ήταν trainable.
  - Ειδικά στα απλά RNN-based μοντέλα, η πτώση ήταν πιο αισθητή.

- **Μικρή βελτίωση του χρόνου εκπαίδευσης** (~0.5–1 sec ανά epoch), πιθανότατα επειδή δεν υπολογίζονταν gradients για το embedding layer.

- **Σημαντική μείωση στον αριθμό παραμέτρων**:
  - Το frozen embedding layer δεν έχει trainable parameters πλέον, οπότε οι συνολικοί παράμετροι του μοντέλου μειώθηκαν δραματικά.
  - Π.χ. `1RNN` είχε μόνο ~10.884 παραμέτρους με frozen embeddings, αντί για ~2 εκατομμύρια όταν ήταν trainable.

---

📌 Γενικό Συμπέρασμα:
> Το "πάγωμα" των embeddings μειώνει την πολυπλοκότητα και το χρόνο εκπαίδευσης, αλλά μπορεί να οδηγήσει σε μικρή πτώση της τελικής ακρίβειας. Το αποτέλεσμα εξαρτάται από το μοντέλο και το πόσο κατάλληλα είναι τα προεκπαιδευμένα embeddings για το συγκεκριμένο task.


## 🎬 Ερώτημα Γ.6 – Εφαρμογή σε νέο Dataset: IMDB Movie Reviews

Σε αυτό το ερώτημα εφαρμόζεται η ίδια διαδικασία με τα προηγούμενα, αλλά σε ένα **νέο dataset**: το IMDB dataset που περιλαμβάνει 50.000 κριτικές ταινιών, ταξινομημένες ως **θετικές** ή **αρνητικές**.


### Εισαγωγή και Προεπεξεργασία Δεδομένων

- Το dataset έχει ληφθεί από [Kaggle](https://www.kaggle.com/datasets/lakshmi25npathi/imdb-dataset-of-50k-movie-reviews) και περιέχει δύο στήλες:
  - `review` → το κείμενο της κριτικής
  - `sentiment` → η ετικέτα (`positive`, `negative`)
  
- Οι ετικέτες μετατράπηκαν σε αριθμούς: `positive = 1`, `negative = 0`

- Το dataset χωρίστηκε σε **training (80%)** και **test (20%)** με `train_test_split` και `stratify` ώστε να διατηρηθεί η ισορροπία των κατηγοριών.


In [191]:
# Φόρτωση του IMDB dataset
dataset_imdb = pd.read_csv("IMDB Dataset.csv")

# Mapping από "positive"/"negative" σε 1/0
dataset_imdb["label"] = dataset_imdb["sentiment"].map({"positive": 1, "negative": 0})

# Split 80/20
from sklearn.model_selection import train_test_split

X_train_imdb, X_test_imdb, y_train_imdb, y_test_imdb = train_test_split(
    dataset_imdb["review"],
    dataset_imdb["label"],
    test_size=0.2,
    random_state=42,
    stratify=dataset_imdb["label"]
)


In [195]:
# Δημιουργία dataset λιστών
train_dataset_imdb = [(label, text) for label, text in zip(y_train_imdb, X_train_imdb)]
test_dataset_imdb = [(label, text) for label, text in zip(y_test_imdb, X_test_imdb)]


### Εκτέλεση Πειραμάτων

- Χρησιμοποιήθηκε **ξεχωριστό λεξιλόγιο** (`vocab_imdb`) που δημιουργήθηκε από τα train/test κείμενα του IMDB.
- Οι προβλέψεις έγιναν με τα 6 προκαθορισμένα μοντέλα (RNN / LSTM, 1-layer / 2-layer, uni/bidirectional).
- Το collate function προσαρμόστηκε με `shift_labels=False` καθώς οι ετικέτες είναι ήδη 0 και 1.


In [196]:
# Χτίσιμο vocab_imdb IMDB
vocab_imdb = build_vocab_from_iterator(
    build_vocabulary([train_dataset_imdb, test_dataset_imdb]),
    min_freq=10,
    specials=["<PAD>", "<UNK>"]
)
vocab_imdb.set_default_index(vocab_imdb["<UNK>"])

print(f"Vocabulary size (IMDB): {len(vocab_imdb)}")


Vocabulary size (IMDB): 29065


In [197]:
from torch.utils.data import DataLoader

train_loader_imdb = DataLoader(train_dataset_imdb, batch_size=BATCH_SIZE, shuffle=True, collate_fn=lambda batch: collate_batch(batch, shift_labels=False))
test_loader_imdb = DataLoader(test_dataset_imdb, batch_size=BATCH_SIZE, shuffle=False, collate_fn=lambda batch: collate_batch(batch, shift_labels=False))


In [198]:
df_imdb_results, preds_imdb = run_experiments(
    configs=model_configs,
    pretrained_embeddings=None,
    freeze_embeddings=False,
    vocab_to_use=vocab_imdb,
    output_dim=2,
    train_loader=train_loader_imdb,
    test_loader=test_loader_imdb
)



🔄 Training: 1RNN
Epoch 1


100%|██████████| 40/40 [00:03<00:00, 10.61it/s]


Train Loss: 0.6943 | Val Loss: 0.6918 | Val Accuracy: 0.5215

Epoch 2


100%|██████████| 40/40 [00:03<00:00, 13.29it/s]


Train Loss: 0.6844 | Val Loss: 0.6728 | Val Accuracy: 0.5850

Epoch 3


...

Train Loss: 0.4565 | Val Loss: 0.5989 | Val Accuracy: 0.7028

Epoch 14


100%|██████████| 40/40 [00:04<00:00,  9.02it/s]


Train Loss: 0.4536 | Val Loss: 0.6015 | Val Accuracy: 0.7013

Epoch 15


100%|██████████| 40/40 [00:05<00:00,  7.28it/s]


Train Loss: 0.4497 | Val Loss: 0.6036 | Val Accuracy: 0.7004



In [206]:
# Transpose results to match the table format in the assignment
df_transposed_imdb = df_imdb_results.set_index("Model").T

# Round accuracy to 2 decimals, format numbers
df_display_imdb = pd.DataFrame({
    col: [
        f"{df_transposed_imdb.loc['Accuracy', col]:.2f}",
        f"{int(df_transposed_imdb.loc['Parameters', col]):,}",
        f"{df_transposed_imdb.loc['Time (sec)', col]:.2f}"
    ]
    for col in df_transposed_imdb.columns
}, index=["Accuracy", "Parameters", "Time cost"])

df_display_imdb

,1RNN,1Bi-RNN,2Bi-RNN,1LSTM,1Bi-LSTM,2Bi-LSTM
Accuracy,69.49,69.72,69.88,69.90,70.46,70.04
Parameters,"2,917,254","2,928,006","2,952,838","2,949,126","2,991,750","3,091,078"
Time cost,3.76,4.18,4.92,3.98,4.29,5.06


### Ερώτημα Γ.56 – Συμπεράσματα

- Όλες οι προσεγγίσεις εμφάνισαν **παρόμοια ακρίβεια**, με ελαφρώς καλύτερες επιδόσεις τα **Bi-Directional μοντέλα**.
- Τα **LSTM** είχαν ελαφρώς καλύτερα αποτελέσματα από τα αντίστοιχα **RNN**, όπως ήταν αναμενόμενο λόγω της πιο εξελιγμένης αρχιτεκτονικής τους.
- Οι **χρόνοι εκπαίδευσης** αυξάνονται αισθητά με την πολυπλοκότητα του μοντέλου (ιδιαίτερα στο `2Bi-LSTM`).
- Αν και τα πιο σύνθετα μοντέλα προσφέρουν μικρή αύξηση στην ακρίβεια, **το απλό `1RNN`** φαίνεται να προσφέρει **καλή ισορροπία** μεταξύ:
  - Απόδοσης
  - Χρόνου εκπαίδευσης
  - Αριθμού παραμέτρων

📌 Συμπερασματικά, **το `1RNN` είναι πιθανώς η βέλτιστη επιλογή στην παρούσα φάση** όταν προτεραιότητα είναι η ταχύτητα και η απλότητα, χωρίς μεγάλη θυσία στην ακρίβεια.
